# Introduction to WaterTAP Costing

This tutorial will demonstrate:
- How to add costing on a flowsheet for reverse osmosis and pump unit models.  
- How to access and update default cost parameters.  
- How to add custom costing relationships.

## Step 1: Import Modules

In [2]:
# Imports from Pyomo
from pyomo.environ import (
    ConcreteModel,
    Var,
    Constraint,
    Objective,
    value,
    assert_optimal_termination,
    TransformationFactory,
    units as pyunits,
)

# Imports from IDAES
from idaes.core import FlowsheetBlock
from idaes.core import UnitModelCostingBlock

# Imports from WaterTAP
from watertap.property_models.seawater_prop_pack import SeawaterParameterBlock
from watertap.unit_models.pressure_changer import Pump
from watertap.unit_models.reverse_osmosis_0D import (
    ReverseOsmosis0D,
    ConcentrationPolarizationType,
    MassTransferCoefficient,
    PressureChangeType,
)
from watertap.costing import WaterTAPCosting
from watertap.costing.unit_models.reverse_osmosis import (
    cost_high_pressure_reverse_osmosis,
)
from watertap.costing.unit_models.pump import cost_high_pressure_pump

## Step 2: Create an RO Model

In [3]:
# Build flowsheet essentials
m = ConcreteModel()
m.fs = FlowsheetBlock(dynamic=False)

# Build property model
m.fs.properties = SeawaterParameterBlock()

# Build unit model
m.fs.RO = ReverseOsmosis0D(
    property_package=m.fs.properties,
    has_pressure_change=True,
    pressure_change_type=PressureChangeType.calculated,
    mass_transfer_coefficient=MassTransferCoefficient.calculated,
    concentration_polarization_type=ConcentrationPolarizationType.calculated,
)

## Step 3: Add Costing

### Step 3a: Add Costing Block to the Flowsheet

`WaterTAPCosting` contains  methods, variables and constraints that are used to calculate flowsheet/system level cost metrics. This sets up the base cost model for the flowsheet, to which unit model specific cost terms will be added.

`WaterTAPCosting` aggregates the unit model specific capital and operating cost terms for all the unit models on the flowsheet.

This is conventionally defined as `m.fs.costing`, and we'll also establish the base currency for the cost model.

<p align="center">
<img src='graphics/cost_framework_1.png' alt="Pump Costing Assumptions" width="400">
</p>

In [4]:
# Set up WaterTAP Costing
m.fs.costing = WaterTAPCosting()

# Set units of base currency
m.fs.costing.base_currency = pyunits.USD_2020

# Looking at the cost variables in m.fs.costing
for var in m.fs.costing.component_objects(ctype=Var, descend_into=False):
    var.display()

total_investment_factor : Total investment factor [investment cost/equipment cost]
    Size=1, Index=None, Units=dimensionless
    Key  : Lower : Value : Upper : Fixed : Stale : Domain
    None :  None :   1.0 :  None :  True : False :  Reals
maintenance_labor_chemical_factor : Maintenance-labor-chemical factor [fraction of equipment cost/year]
    Size=1, Index=None, Units=1/a
    Key  : Lower : Value : Upper : Fixed : Stale : Domain
    None :  None :  0.03 :  None :  True : False :  Reals
utilization_factor : Plant capacity utilization [fraction of uptime]
    Size=1, Index=None, Units=dimensionless
    Key  : Lower : Value : Upper : Fixed : Stale : Domain
    None :  None :   0.9 :  None :  True : False :  Reals
electricity_cost : Electricity cost
    Size=1, Index=None, Units=USD_2018/kWh
    Key  : Lower : Value : Upper : Fixed : Stale : Domain
    None :  None :  0.07 :  None :  True : False :  Reals
electrical_carbon_intensity : Grid carbon intensity [kgCO2_eq/kWh]
    Size=1, 

### Step 3b: Add RO Unit Model Costing Block

Next, `UnitModelCostingBlock` is used to add the unit model specific costing model to the flowsheet. This will set up all the unit specific variables and constraints related to costing the unit.

**Note:** `UnitModelCostingBlock` requires the configuration argument `flowsheet_costing_block` to be set which refers to the instance of flowsheet level costing block (usually `m.fs.costing`)

<p align="center">
<img src='graphics/cost_framework_2.png' alt="Pump Costing Assumptions" width="400">
</p>

In [5]:
# Create RO costing block
m.fs.RO.costing = UnitModelCostingBlock(
    flowsheet_costing_block=m.fs.costing,
)

The `UnitModelCostingBlock` also adds variables for capital and operating cost to the unit model as shown below.

**Note**: The variable operating cost which could include chemical flows and electricity flows are aggregates at the flowsheet level.

In [6]:
# Display the capital and fixed operating cost variables for RO
m.fs.RO.costing.capital_cost.display()
m.fs.RO.costing.fixed_operating_cost.display()

capital_cost : Unit capital cost
    Size=1, Index=None, Units=USD_2020
    Key  : Lower : Value    : Upper : Fixed : Stale : Domain
    None :     0 : 100000.0 :  None : False : False : NonNegativeReals
fixed_operating_cost : Unit fixed operating cost
    Size=1, Index=None, Units=USD_2020/a
    Key  : Lower : Value    : Upper : Fixed : Stale : Domain
    None :     0 : 100000.0 :  None : False : False : NonNegativeReals


## Step 4: Display the Cost Model Components

Reverse osmosis unit model cost variables are now added to the flowsheet costing block `m.fs.costing` and can be accessed as shown.

Now all instances of the reverse osmosis model on the flowsheet will share share the same global parameters.

In [7]:
# Print the cost parameters for reverse osmosis
m.fs.costing.reverse_osmosis.display()

Block fs.costing.reverse_osmosis

  Variables:
    factor_membrane_replacement : Membrane replacement factor [fraction of membrane replaced/year]
        Size=1, Index=None, Units=1/a
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :  None :   0.2 :  None :  True : False :  Reals
    membrane_cost : Membrane cost
        Size=1, Index=None, Units=USD_2018/m**2
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :  None :    30 :  None :  True : False :  Reals
    high_pressure_membrane_cost : Membrane cost
        Size=1, Index=None, Units=USD_2018/m**2
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :  None :    75 :  None :  True : False :  Reals

  Objectives:
    None

  Constraints:
    None


**Note:** The block name for the reverse osmosis block added to the flowsheet is `reverse_osmosis`. 

This is because, the cost block name defined in the costing model for RO is `reverse_osmosis`. You can always check how it is defined by looking at the costing model or by looking at the `m.fs.costing` flowsheet costing block like we will do below.

In [8]:
m.fs.costing.display()

Block fs.costing

  Variables:
    total_investment_factor : Total investment factor [investment cost/equipment cost]
        Size=1, Index=None, Units=dimensionless
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :  None :   1.0 :  None :  True : False :  Reals
    maintenance_labor_chemical_factor : Maintenance-labor-chemical factor [fraction of equipment cost/year]
        Size=1, Index=None, Units=1/a
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :  None :  0.03 :  None :  True : False :  Reals
    utilization_factor : Plant capacity utilization [fraction of uptime]
        Size=1, Index=None, Units=dimensionless
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :  None :   0.9 :  None :  True : False :  Reals
    electricity_cost : Electricity cost
        Size=1, Index=None, Units=USD_2018/kWh
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :  None :  0.07 :  None :  True :

## Step 5: Modify Existing Parameters

Let's say we want to change the membrane unit cost for the reverse osmosis model from 30 USD/m<sup>2</sup> to 60 USD/m<sup>2</sup>. 

The cost variable name is `membrane_cost`. This variable was added with the unit model cost block `reverse_osmosis` to the flowsheet cost block `m.fs.costing`

In [9]:
# Update RO membrane cost
m.fs.costing.reverse_osmosis.membrane_cost.fix(60)

# Print the cost model for RO
m.fs.costing.reverse_osmosis.display()

Block fs.costing.reverse_osmosis

  Variables:
    factor_membrane_replacement : Membrane replacement factor [fraction of membrane replaced/year]
        Size=1, Index=None, Units=1/a
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :  None :   0.2 :  None :  True : False :  Reals
    membrane_cost : Membrane cost
        Size=1, Index=None, Units=USD_2018/m**2
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :  None :    60 :  None :  True : False :  Reals
    high_pressure_membrane_cost : Membrane cost
        Size=1, Index=None, Units=USD_2018/m**2
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :  None :    75 :  None :  True : False :  Reals

  Objectives:
    None

  Constraints:
    None


## Try It Yourself

Add a `UnitModelCostingBlock` for a pump and change the cost from 1.908 USD/W to 1 USD/W given the following information.

In [10]:
# Build pump unit model
m.fs.pump = Pump(
    property_package=m.fs.properties,
)

# Add pump UnitModelCostingBlock
m.fs.pump.costing = UnitModelCostingBlock(
    flowsheet_costing_block=m.fs.costing,
)

# Print the cost parameters for the pump before updating
m.fs.costing.high_pressure_pump.display()

# Update pump cost parameter
m.fs.costing.high_pressure_pump.cost.fix(1)

# Print the cost model for the pump to verify changes
m.fs.costing.high_pressure_pump.display()

Block fs.costing.high_pressure_pump

  Variables:
    cost : High pressure pump cost
        Size=1, Index=None, Units=USD_2018/W
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 : 1.908 :  None :  True : False :  Reals

  Objectives:
    None

  Constraints:
    None
Block fs.costing.high_pressure_pump

  Variables:
    cost : High pressure pump cost
        Size=1, Index=None, Units=USD_2018/W
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :     1 :  None :  True : False :  Reals

  Objectives:
    None

  Constraints:
    None


In [11]:
m.fs.costing.display()

Block fs.costing

  Variables:
    total_investment_factor : Total investment factor [investment cost/equipment cost]
        Size=1, Index=None, Units=dimensionless
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :  None :   1.0 :  None :  True : False :  Reals
    maintenance_labor_chemical_factor : Maintenance-labor-chemical factor [fraction of equipment cost/year]
        Size=1, Index=None, Units=1/a
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :  None :  0.03 :  None :  True : False :  Reals
    utilization_factor : Plant capacity utilization [fraction of uptime]
        Size=1, Index=None, Units=dimensionless
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :  None :   0.9 :  None :  True : False :  Reals
    electricity_cost : Electricity cost
        Size=1, Index=None, Units=USD_2018/kWh
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :  None :  0.07 :  None :  True :

Another optional configuration arguments on `UnitModelCostingBlock` can be set is `costing_method`, which allows the user to choose between different cost assumptions. 

As shown  below, RO has two costing methods:

<p align="center">
<img src='graphics/pump_costing_doc.png' alt="Pump Costing Assumptions" width="1000">
</p>

## Step 6: Add System Level Cost Metrics

The cost process function creates the variables that are used for calculating system level cost metrics:
- `aggregate_capital_cost`
- `aggregate_fixed_operating_cost`
- `aggregate_variable_operating_cost`
- `aggregate_flow_costs`
- `total_capital_cost`
- `total_operating_cost`

<p align="center">
<img src='graphics/cost_framework_3.png' alt="Pump Costing Assumptions" width="1000">
</p>

In [12]:
# Create aggregate costing variables by using cost_process
m.fs.costing.cost_process()

# We need to initialize the aggregate costing variables that were created as a part of the cost_process step
m.fs.costing.initialize()
# Add levelized cost of water
m.fs.costing.add_LCOW(m.fs.RO.mixed_permeate[0].flow_vol_phase['Liq'])
# Add specific energy consumption (SEC)
m.fs.costing.add_specific_energy_consumption(
    m.fs.RO.mixed_permeate[0].flow_vol_phase['Liq'],
)

<p align="center">
<img src='graphics/cost_framework_4.png' alt="Pump Costing Assumptions" width="600">
</p>

In [13]:
# Display costing results
print("\nCosting Results:")
print(f"Total Capital Cost: {value(m.fs.costing.total_capital_cost):.2f} USD/year")
print(f"Total Operating Cost: {value(m.fs.costing.total_operating_cost):.2f} USD/year")
print(f"Levelized Cost of Water (LCOW): {value(m.fs.costing.LCOW):.2f} USD/m3")
print(f"Specific Energy Consumption (SEC): {value(m.fs.costing.specific_energy_consumption):.2f} kWh/m3")


Costing Results:
Total Capital Cost: 1186.27 USD/year
Total Operating Cost: 154.22 USD/year
Levelized Cost of Water (LCOW): 0.00 USD/m3
Specific Energy Consumption (SEC): 0.00 kWh/m3


## Try it Yourself

In [14]:
# Display the capital cost for pump 
m.fs.pump.costing.capital_cost.display()

# Display the capital cost for pump 
m.fs.RO.costing.capital_cost.display()

capital_cost : Unit capital cost
    Size=1, Index=None, Units=USD_2020
    Key  : Lower : Value : Upper : Fixed : Stale : Domain
    None :     0 :   0.0 :  None : False : False : NonNegativeReals
capital_cost : Unit capital cost
    Size=1, Index=None, Units=USD_2020
    Key  : Lower : Value              : Upper : Fixed : Stale : Domain
    None :     0 : 1186.2709335101972 :  None : False : False : NonNegativeReals


## Step 7: Register Custom Flows

Could walk through how they might add their own costing (register their own flow types) as shown in #2 here: https://watertap.readthedocs.io/en/latest/technical_reference/costing/costing_base.html, but after some further reflection, I think this is best left for a post-session tutorial. This tutorial functions as a good introduction as it currently stands.